In [1]:
import pandas as pd
import csv
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import joblib
from src.util import *

C:\Users\Samue\Anaconda3\envs\DS-U4-S1-NLP\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
submission_table = pd.read_csv('top_submission_info (1).csv')
submission_table.head()

,Unnamed: 0,title,score,text,nsfw,subreddit_name
0,0,"People who haven't pooped in 2019 yet, why are...",221863,NaN,False,AskReddit
1,1,Stan Lee has passed away at 95 years old,175334,As many of you know today is day that many of ...,False,AskReddit
2,2,"Reddit, how would you feel about a law that ba...",160308,NaN,False,AskReddit
3,3,"Without saying what the category is, what are ...",144658,,False,AskReddit
4,4,"Anthony Bourdain once said ""There's a guy in m...",141143,NaN,False,AskReddit


In [3]:
submission_table = submission_table.drop(columns='Unnamed: 0')

In [4]:
submission_table.head()

,title,score,text,nsfw,subreddit_name
0,"People who haven't pooped in 2019 yet, why are...",221863,NaN,False,AskReddit
1,Stan Lee has passed away at 95 years old,175334,As many of you know today is day that many of ...,False,AskReddit
2,"Reddit, how would you feel about a law that ba...",160308,NaN,False,AskReddit
3,"Without saying what the category is, what are ...",144658,,False,AskReddit
4,"Anthony Bourdain once said ""There's a guy in m...",141143,NaN,False,AskReddit


In [5]:
# def tokenize(doc):
#     return [token for token in simple_preprocess(doc) if token not in STOPWORDS]

In [6]:
tfidf = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 2))
sparse = tfidf.fit_transform(submission_table['title'])

In [7]:
joblib.dump(tfidf, 'tfvect.joblib')

['tfvect.joblib']

In [8]:
dtm = pd.DataFrame(sparse.todense(), columns=tfidf.get_feature_names())
dtm.head()

,aa,aa batteries,aaa,abandon,abandon twins,abandoned,abandoned walmart,abandonment,abandonment terry,abducted,...,ｃｈａｎｇｅｄ ｍｙ,ｆｏｎｔ,ｆｏｎｔ ｔｏ,ｍｙ,ｍｙ ｆｏｎｔ,ｔｈｉｓ,ｔｏ,ｔｏ ｔｈｉｓ,ｖｅ,ｖｅ ａｃｃｉｄｅｎｔａｌｌｙ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
nn = NearestNeighbors(n_neighbors=5, algorithm='brute')
nn.fit(dtm)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [10]:
joblib.dump(nn, 'nn.joblib')

['nn.joblib']

In [11]:
query = ["""This cat's peculiar jumping style."""]

In [12]:
looking = tfidf.transform(query)
nn.kneighbors(looking.todense())

(array([[0., 1., 1., 1., 1.]]),
 array([[1675,  567, 1344, 1180,  562]], dtype=int64))

In [13]:
for i in nn.kneighbors(looking.todense())[1]:
    print(submission_table['subreddit_name'][i])

1675    oddlysatisfying
567          Unexpected
1344       HistoryMemes
1180       comedyheaven
562          Unexpected
Name: subreddit_name, dtype: object
